In [1]:
#Libraries
%matplotlib inline
import sys

import numpy as np

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import datetime
import time

sns.set()
pal = sns.hls_palette(10 , h=0.5)
sns.set_palette(pal)

/Users/shameikdutta/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_uber = pd.read_csv('uber_nyc_data.csv')

In [3]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   origin_taz       object 
 2   destination_taz  object 
 3   pickup_datetime  object 
 4   trip_distance    float64
 5   trip_duration    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ GB


In [4]:
df_uber

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
0,252581,7C,6A,2014-09-01 09:00:00,4.25,0:15:11
1,252582,7B,15,2014-09-01 18:00:00,10.17,0:34:05
2,252583,11,2A,2014-09-01 17:00:00,4.02,0:17:06
3,252584,3B,4A,2014-09-01 13:00:00,1.46,0:06:32
4,252585,2A,10,2014-09-01 14:00:00,8.31,0:26:17
...,...,...,...,...,...,...
30925733,30693431,15,15,2015-09-01 00:00:00,6.86,0:27:37
30925734,30693447,5B,2A,2015-09-01 00:00:00,3.44,0:11:07
30925735,30693461,4C,NaN,2015-09-01 00:00:00,6.17,0:19:10
30925736,30693466,2A,5A,2015-09-01 00:00:00,1.87,0:07:54


In [5]:
df_uber.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
0,252581,7C,6A,2014-09-01 09:00:00,4.25,0:15:11
1,252582,7B,15,2014-09-01 18:00:00,10.17,0:34:05
2,252583,11,2A,2014-09-01 17:00:00,4.02,0:17:06
3,252584,3B,4A,2014-09-01 13:00:00,1.46,0:06:32
4,252585,2A,10,2014-09-01 14:00:00,8.31,0:26:17


# Check for missing data

In [6]:
len(df_uber[df_uber.duplicated() == True])
#no duplicate data in the rows

0

In [7]:
arr1 = df_uber.origin_taz.unique()
arr1

array(['7C', '7B', '11', '3B', '2A', '5B', '10', '2B', '9', '6B', '15',
       '4C', '5A', '8', '14', '4A', '7A', '1', '16', '3A', '2C', '6A',
       '12', '13', '3C', '4B', '5C', '17'], dtype=object)

In [8]:
arr2 = df_uber.destination_taz.unique()
arr2

array(['6A', '15', '2A', '4A', '10', '4C', '7A', '3C', '5B', '14', '8',
       nan, '7C', '12', '2C', '1', '6B', '5C', '9', '3A', '2B', '11',
       '7B', '5A', '13', '4B', '18', '16', '3B', '17'], dtype=object)

In [9]:
set(arr2) - set(arr1)

{'18', nan}

In [10]:
df_uber[df_uber.destination_taz.isnull()].head()
#check mdata with missing destination

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15,252596,6B,NaN,2014-09-01 17:00:00,11.67,0:23:45
32,252613,9,NaN,2014-09-01 16:00:00,80.69,1:59:37
63,252644,9,NaN,2014-09-01 18:00:00,23.07,1:03:37
141,252722,9,NaN,2014-09-01 10:00:00,21.73,0:41:12
148,252729,4B,NaN,2014-09-01 10:00:00,4.62,0:14:55


In [11]:
df = df_uber[df_uber.trip_duration.isnull() & df_uber.trip_distance.isnull()]
df.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15155317,39535,2A,11,2015-04-25 12:00:00,NaN,NaN
15245057,1009076,2A,2A,2015-04-26 01:00:00,NaN,NaN
16519652,15028665,2A,7C,2015-04-29 21:00:00,NaN,NaN
17148253,22250173,2A,2A,2015-04-12 02:00:00,NaN,NaN
17297563,23716998,2C,11,2015-04-25 13:00:00,NaN,NaN


In [12]:
len(df_uber[(df_uber.trip_duration.isnull() == False) & (df_uber.trip_distance.isnull())])
##All cases for which trip_duration is null match the cases for which trip_distance is also null:

0

# Analyze the pickup_datetime column to datetime type and create related features.

In [13]:
len(df_uber[df_uber.pickup_datetime.apply(lambda x:x[-5:] == '00:00') == False])
#pickup time data offer mins or sec precision

0

In [14]:
def dateParser(s): # takes a string in format yyy-mm-dd hh:mm:ss and returns the same as a datetime object
    
    return datetime.datetime(int(s[0:4]) , int(s[5:7]) , int(s[8:10]) , int(s[11:13]))

In [15]:
t0 = time.time()
df_uber['pu_date_hour'] = df_uber.pickup_datetime.apply(dateParser)
time.time() - t0

36.92399787902832

In [16]:
beginning = df_uber.pu_date_hour.min()
end = df_uber.pu_date_hour.max()
print(beginning, end, end - beginning)

2014-09-01 00:00:00 2015-09-01 00:00:00 365 days 00:00:00


In [17]:
df_uber = df_uber.drop('pickup_datetime' , axis = 1) #drop coulmns axis  = 1 means coulmns

In [ ]:
t0 = time.time()
df_uber['pu_date'] = pd.Series(map(lambda x: x.astype('datetime64[D]'),df_uber['pu_date_hour'].values))
time.time() - t0
#creation of coulmn with the date and without time

In [ ]:
df_uber['year'] = df_uber['pu_date_hour'].dt.year
df_uber['month'] = df_uber['pu_date_hour'].dt.month
#get month and year from pickup stamp

In [ ]:
t0 = time.time()
df_uber['hour'] = df_uber['pu_date_hour'].dt.hour
time.time() - t0
#get trip pick up hour from timestamp

In [ ]:
t0 = time.time()
df_uber['weekday'] = df_uber['pu_date_hour'].dt.dayofweek
time.time() - t0
#trip pickup weekday from timestamp

In [ ]:
df_uber.weekday.value_counts()
#counts as monday = 0 ----- Sunday = 6

In [ ]:
df_uber.head()

# Transformation of trip duration data

In [ ]:
uniq_dur = df_uber[df_uber.trip_duration.isnull() == False].trip_duration.unique()

In [ ]:
#how many represent >=10h of duration
long_duration = []
for item in uniq_dur:
    if len(item) != 7:
        long_duration.append(item)
print(len(long_duration))

In [ ]:
#check for unusual string duration
for item in long_duration:
    if len(item) > 8:
        print(item)

In [ ]:
def duration_to_minutes(s):  # Takes hh:mm:ss and returns integer equivalent of total time in mins or zero from data
    if pd.isnull(s):
        val = 0
    else:  
        hms = s.split(':')
        val = int(hms[0]) * 60 + int(hms[1]) + int(hms[2]) / 60.0
    return val  # Return outside the conditional block

In [ ]:
#trip duration string into trip duration into mins
t0 = time.time()
df_uber['duration_min'] = df_uber.trip_duration.apply(duration_to_minutes) 
time.time() - t0

In [ ]:
df_uber.head()

# replace missing values with mean distance and duration for origin to destination

In [ ]:
#mean distance and duration for each origin-destination pair
df_DistDur = df_uber.groupby(['origin_taz' , 'destination_taz'])[['trip_distance' , 'duration_min']].mean()

In [ ]:
df_DistDur.head()

In [ ]:
#replacing missing values with avg distance and duration for the origin-dest pair

for i in df.index:
    orig = df_uber.loc[i , 'origin_taz']
    dest = df_uber.loc[i , 'destination_taz']
    df_uber.loc[i , 'trip_distance'] = df_DistDur.loc[orig , dest].trip_distance
    df_uber.loc[i , 'duration_min'] = df_DistDur.loc[orig , dest].duration_min

# Calculating Avg speed for each trip

In [ ]:
#calculate avg speed 
df_uber['trip_mph_avg'] = df_uber.trip_distance/(df_uber.duration_min/60.0)

In [ ]:
df_uber.head()

# calculating estimated revenue per trip

In [ ]:
#Source: http://uberestimate.com/prices/New-York-City/ for Uber X

base_fare = 2.55
per_minute = 0.35
per_mile = 1.75
min_fare = 8

In [ ]:
#def est_revenue(arr):
#    rev = base_fare + arr[0] * per_minute + arr[1] * per_mile
#    return rev if rev > min_fare else min_fare

In [ ]:
#df_uber['est_revenue'] = pd.Series(map(lambda x: est_revenue(x), test))

In [ ]:
#Using eval() is even more efficient for large datasets than resorting to Numpy! 
#Local variables must be followed by @. Only for arithmetic, cannot use if statement, for example.
df_uber['est_revenue'] = df_uber.eval('@base_fare + duration_min * @per_minute + trip_distance * @per_mile')
df_uber.loc[df_uber.est_revenue < 8, 'est_revenue'] = min_fare

In [ ]:
df_uber.head()

# Checking data consistency

In [ ]:
#With such a large amount of data it is very difficult to go through all the data and find the anomalies.The possible anomilies have been attended and have been worked upon

In [ ]:
#Given below the possible anomalies visible - 1. Cases with distance equal to zero but duration is greater than zero 2.Anomalies with average trip speed

In [ ]:
#Check cases with distance equal to zero but duration greater than zero:

print (len(df_uber[(df_uber.duration_min > 0) & (df_uber.trip_distance == 0)]))

In [ ]:
#checking small dataset with duration > 5 mins , but distance equal to zero

df_uber[(df_uber.duration_min >= 5) & (df_uber.trip_distance == 0) &(df_uber.origin_taz != df_uber.destination_taz)]\
.head(10)

# Anomalies with average trip speed

In [ ]:
#checking trips with speed slower than average walking speed(3mph)

print (len(df_uber[df_uber.trip_mph_avg <=3]))

In [ ]:
#possiblity of traffic so distance travelled less than 2 miles

df_uber[(df_uber.trip_mph_avg <= 3) & (df_uber.trip_distance <=2 )].head(10)

# filtering data definig function and loading methods to create visualizations

In [ ]:
#Create dataframe to be used for visualization with exactly 365 days of data, and max trip duration of 16h:
df_viz = df_uber[(df_uber.pu_date != datetime.date(2015, 9, 1)) & (df_uber.duration_min <= 960)].copy()

In [ ]:
#Descriptive statistics for numerical features:
df_viz[['duration_min', 'trip_distance', 'trip_mph_avg', 'est_revenue']].describe()

In [ ]:
#exporting plotly:
import matplotlib as mlab
import plotly as py

In [ ]:
pip install matplotlib==3.1.3

In [ ]:
from matplotlib.ticker import FuncFormatter #calling formatter function
from matplotlib.offsetbox import (OffsetImage , AnnotationBbox) #creating image box
from matplotlib._png import read_png #load png
from matplotlib.patches import Ellipse #draw ellipse

In [ ]:
def thousands_comma(x,pos):#args are the values and tick positions returns number with thousands comma and no decimals
    return '{:,.0f}'.format(x)

In [ ]:
def thousands_format(x,pos):#args are the value and tick position , returns number of thousands with one decimal and K in place of 3 zeros
    return '{:.0f}{}'.format(x * 1e-6,'K')

In [ ]:
def millions_format(x,pos): #args are the value and tick position , returns number of millions with one decimal and M in place of 6 zeros
    return '{:.1f{}}'.format(x * 1e-6 , 'M')

In [ ]:
def millions_currency(x,pos): #args are the value and tick position , returns number of millions with RS. sign and M in place of 6 zeros
    return '{}{:.0f}{}'.format('Rs',x * 1e-6 , 'M')

# Visualizing the number of Uber trips per day

In [ ]:
#Create a plotting data frame with counts (number of trips ) grouped by day:
byDate = df_viz.groupby('pu_date')['id'].count()
byDate.head()

In [ ]:
fig = plt.figure()
ax = byDate.plot(figsize = (16,8),fontsize = 12 , ylim = (10000,170000),color = 'navy')

# Efftect of time on demand for uber rides:Distribution per hour,weekday and month

In [ ]:
weekday_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month_labels = ['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']

In [ ]:
# Create plots comparing demand per different time categories:
# fig = plt.figure(figsize = (15,16))
# formatter = FuncFormatter(millions_format)

# plt.subplot(3,1,1) #this plot could have a polar format, although I'm not sure it'd be a better viz.
# ax1 = sns.countplot(df_viz.hour, order = range(24))
# ax1.yaxis.set_major_formatter(formatter)
# plt.tick_params(labelsize=13)
# plt.title('Total Number of Trips Between 9/1/2014 and 8/31/2015', fontsize=17, color='navy')
# plt.xlabel('Hour', fontsize=13)

# plt.subplot(3,1,2)
# ax2 = sns.countplot(df_viz.weekday, order = range(7))
# ax2.set_xticklabels(weekday_labels)
# ax2.yaxis.set_major_formatter(formatter)
# plt.tick_params(labelsize=13)
# plt.xlabel('')

# plt.subplot(3,1,3) #same as the line plot but with less granularity, perhaps easier to visualize the MOM growth
# ax3 = sns.countplot(df_viz.month)
# ax3.set_xticklabels(month_labels)
# ax3.yaxis.set_major_formatter(formatter)
#plt.text(0, 3100000, 'Consistent growth in the number of trips from month to month!', color='navy', size=15)
# plt.tick_params(labelsize=13)
# plt.xlabel('')

#fig.subplots_adjust(hspace=.5)
# plt.show()


# Estimated monthly base revenue:How much was the nyc market worth in that period of time

In [ ]:
#Bugs graph wrong

In [ ]:
#creating a dataframe with grouped revenue data:

#byDateRev = df_viz.groupby('month')['est_revenue'].sum()
last_3_months = df_viz[df_viz['month'].isin(df_viz['month'].unique()[-3:])]
byDateRev = last_3_months.groupby('month')['est_revenue'].sum()

In [ ]:
#estimate ubers's gross margin for entire 1 year based on 25% share of the the base revenue
sumRev = byDateRev.sum()
print ('Rs{:,.0f}M'.format(sumRev * 1e-6), "Base Revenue   Uber's Base Gross Margin:", 'Rs{:,.0f}'.format(sumRev * .25))

In [ ]:
#esitmated base gross margin based on information that average Uber fare in sep 2014 was Rs.27
len(df_viz) * 27 * .25

In [ ]:
byDateRev.head()

In [ ]:
#create chart showing total base revenue per month
months_seq = [9,10,11,12,1,2,3,4,5,6,7,8]
formatter = FuncFormatter(millions_currency)

In [ ]:
plt.figure(figsize=(10, 6)) 
ax = sns.barplot(x=byDateRev.index, y=byDateRev.values, order=months_seq)
ax.set_xticklabels(month_labels, rotation=45, ha='right')  # Rotate and align for readability
ax.set_ylabel('Revenue (Millions)', fontsize=15)
ax.yaxis.set_major_formatter(formatter)
plt.title('Estimated Monthly Base Revenue from Sep-2014 to Aug-2015', fontsize=17, color='navy')
total_revenue_text = "Total Base Revenue for the Period: Rs.{:,.0f} Million".format(sumRev * 1e-6)
plt.text(0.7, 0.95, total_revenue_text, ha='left', va='center', fontsize=14, fontweight='bold', color='navy')
plt.tick_params(labelsize=13)
plt.show()


In [ ]:
#Mean fare from revenue estimate for the period
df_viz.est_revenue.mean()

In [ ]:
#estimation of revenue share from Uber from 12/1/14 to 8/31/15

rev = df_viz[(df_viz.pu_date.dt.date >= datetime.date(2014, 12, 1)) & (df_viz.pu_date.dt.date <= datetime.date(2015, 8, 31))] \
    .est_revenue.mean() 

u_pool = rev * 0.75 * 1500000
u_pool 

In [ ]:
u_pool / df_viz[(pd.to_datetime(df_viz.pu_date).dt.date >= datetime.date(2014, 12, 1)) & (pd.to_datetime(df_viz.pu_date).dt.date <= datetime.date(2015, 8, 31))] \
    .est_revenue.sum()

# Month over Month growth revenue growth: How fast has Uber grown at that peroid of time

In [ ]:
#Lots of bugs to be entertained later

In [ ]:
#Create dictionary with calculated month over month revenue percentage growth:
dict_growth = {}
for i in range(len(months_seq)):
    mo = months_seq[i]
    rev = byDateRev[mo]
    if mo == 9:
        growth_pct = 0
    elif mo == 1:
        growth_pct = ((rev/byDateRev[12]) - 1) * 100
    else:
        growth_pct = ((rev/byDateRev[mo - 1]) - 1) * 100
    print(month_labels[i], 'Revenue =', 'Rs{:,.0f}'.format(rev), 'Growth % = ', '{:.1f}'.format(growth_pct))


# Day of week has the slowest traffic overall

In [ ]:
#create dataframe with grouped data.
df_plt2 = df_viz.groupby('weekday')['trip_mph_avg'].median()

In [ ]:
#creating plot

fig = plt.figure()
albels_list = df_plt2.values.round(1)
kwargs = {'fontsize':12 , 'ha':'center' , 'va':'bottom','weight':'bold','color':'navy'}
ax = df_plt2.plot(kind='bar', rot = 0, figsize = (12, 6), fontsize = 12, edgecolor='yellow', linewidth=1)
ax.set_xticklabels(weekday_labels, weight='bold')

ax.get_yaxis().set_ticks([])
#annotate_labels(ax, labels_list, **kwargs)

plt.xlabel('')
plt.ylabel('Median Trip Speed (mph)', fontsize = 14)
plt.tick_params(labelsize=14)
plt.title('Which day of the week has the worst traffic?', fontsize = 18, color='navy')
plt.show()

In [ ]:
#upper value not visible

# Median distance travelled per trip?

In [ ]:
plt.figure(figsize = (16,6))

plt.subplot(1,2,1)
df_viz[(df_viz.weekday >= 5)].groupby('hour')['trip_distance'].median().plot(kind='bar', rot = 0, fontsize=14)
plt.ylim(0, 9)
plt.tick_params(labelsize=12)
plt.ylabel('Median Trip Distance (miles)', fontsize=14, weight='bold', color='navy')
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.title('Weekend', fontsize=14, weight='bold', color='navy')

plt.subplot(1,2,2)
df_viz[df_viz.weekday < 5].groupby('hour')['trip_distance'].median().plot(kind='bar', rot = 0)
plt.tick_params(labelsize=12)
plt.xlabel('Hour', fontsize=14, weight='bold', color='navy')
plt.title('Weekdays', fontsize=14, weight='bold', color='navy')

plt.show()

# What are the most popular pickup and drop off taxi zones